# Import packages

In [ ]:
import sys
sys.path.append('../PyNNcml/') # This line is need to import pynncml
import torch
import pynncml as pnc
import numpy as np
from matplotlib import pyplot as plt
import opensense_data_downloader_and_transformer as oddt

C:\Users\haih\Anaconda3\envs\opensense\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download Dataset 
This is a generic code that download and preprocess the OpenMRG dataset.

In [ ]:
oddt.download_andersson_2022_OpenMRG(local_path='data/andersson_2022_OpenMRG/', print_output=True)
ds = oddt.transform_andersson_2022_OpenMRG(
    fn='data/andersson_2022_OpenMRG/OpenMRG.zip', 
    path_to_extract_to='data/andersson_2022_OpenMRG/',
)

File already exists at desired location data/andersson_2022_OpenMRG/OpenMRG.zip
Not downloading!


# Transform Dataset to PyNNcml format
In this part of the code, we transform the OpenMRG dataset into PyNNcml format. In addition, we generate a Min-Max 15 minutes data from TSL and RSL  every  10sec sample to illustrate the use of some algorithms inside PyNNcml.

In [ ]:
sublink_index=0
ds_sublink=ds.isel(sublink=sublink_index)
# frequency, polarization, length, height_far, height_near
meta_data=pnc.MetaData(float(ds_sublink.frequency),
                   bool(ds_sublink.polarization!='Vertical'), # Vertical 1
                   float(ds_sublink.length),
                   25,
                   25)
rsl_array=np.asarray(ds_sublink.rx)
tsl_array=np.asarray(ds_sublink.tx)
time_array=np.asarray(ds_sublink.time).astype('datetime64[s]').astype("int")
link_data=pnc.Link(rsl_array,
                   np.zeros(time_array.shape[0]), # Insert zero instade of rain rate values.
                   time_array,
                   meta_data,
                   tsl_array)
link_min_max=link_data.create_min_max_link(300)
att=link_min_max.attenuation().T
att=torch.unsqueeze(att,dim=0)

# Running Dynamic Baseline +Power Law
In the following line of code: we run a dynamic baseline algorithm [1] which is followed by calibrated min-max power-law [2].

In [ ]:
osd = pnc.rain_estimation.one_step_dynamic_baseline(pnc.power_law.PowerLawType.MAX,1.2,8) # init classification model
rain = osd(att,link_min_max.meta_data) # run Dynamic method

# Plot
Here, we plot the rain estimation and sub-link min-max attenuation 

In [ ]:

plt.subplot(1,2,1)
plt.plot(rain[0,:].numpy())
plt.show()
plt.subplot(1,2,2)
# plt.plot(wd)

# References 
[1] 

[2]